In [2]:
import tensorflow as tf

from tensorflow.python.keras.layers import Layer, Conv2D
from tensorflow.python.keras.losses import SparseCategoricalCrossentropy

In [5]:
from prune_test import load_model, load_dataset, compile_model, train_model
from pruner import Pruner

ModuleNotFoundError: No module named 'pruning'